In [ ]:
import os.path as op
import pandas as pd
import glob

In [ ]:
def find_datafiles(filename):
    f = glob.glob(filename)
    return(f)

In [ ]:
data_folder = "C:\\Users\\LocalAdmin\\Documents\\GUESS\\Guess_fMRI\\data"

In [ ]:
# find renamed data files to identify relevant log files and obtain ID
filename = op.join(data_folder, 'pilot2_*.csv')
file_list = find_datafiles(filename)
frames = []
print(filename)
print(file_list)
# loop through file list
for f in file_list:
    split_fname = f.split('_') # split name into chunks seperated by '_'
    pid = split_fname[1].split('\\')[-1] # second chunk is the ID
    date = split_fname[-2] # second to last bit is date
    time = split_fname[-1].split('.') # last bit is time + extension
    time.pop()
    time = '.'.join(time)
    
    # find log file matching date and time
    log_fname = pid + "_guess_" + date + '_' + time + ".log" # pilot2_guess_2025-04-22_13h19.30.422.log
    log_fpath = op.join(data_folder, log_fname)
    log_file_list = find_datafiles(log_fpath)
    print(log_file_list)
    if len(log_file_list) > 1:
        print('Warning! More than 1 log file.')
    elif len(log_file_list) < 1:
        print('Warning: No log file attached to data sheet.')
        print('\t affected data: ', date, time, id)
        continue
    log_file = log_file_list[0] # open the first occurence
    log_file = open(log_file, "r")
    log = log_file.read()
    log_part1_part2 = ''
    
    if "end_part2_text: autoDraw = True" in log: # check if participant finished the task
        log_part1_part2 = log.split('end_part1_text: autoDraw = True')[0] #split learning and recall
        if 'start_task_text: autoDraw = True' in log_part1_part2: 
            log_part1_part2 = log_part1_part2.split('start_task_text: autoDraw = True')
            log_part1_part2 = log_part1_part2[1] # we don't care about practice
        trials = log_part1_part2.split("cue_stim_learning: text = ")
        trials.pop(0) # first element is irrelevant, because it precedes the occurence of the word
        print(len(trials)) # sanity check: should be 100 if they finished

        indeces = []
        cues = []
        
        for trial in trials:
            lines = trial.split('\n')
            cue = lines[0] # first line contains the cue
            cues.append(cue)
            indeces.append()
            i += 1
        
        # create df for the participant
        data = np.array([indeces, rts, responses]).transpose()
        column_values = ['Trial', 'RT', 'Response']
        # creating the dataframe
        df = pd.DataFrame(data = data,  
                          columns = column_values)
        df['RT'] = df['RT'].astype(float)
        df['Response'] = df['Response'].astype(float) # save responses as float because of NaN values
        df['Online_ID'] = online_id
        df['Date'] = date
        df['Time'] = time
        df = pd.concat([df, trial_info], axis=1)
        
        df['correct'] = df['Response'] == df['correct_response']
        
        # Responses can be: 1, 2, 3, 4 
        # The difference between a given response and Target is saved in the 'var1_diff' 'var2_diff' etc columns.
        # Make a list from the 'var1_diff'... columns. 
        # The Response can be used as an index, selecting from the right column ((converted to int)-1)
        # Select the row based on the index of the trial
        # If Response is NaN, Diff should be NaN too 
        diff_list = [trial_info['var1_diff'], trial_info['var2_diff'], trial_info['var3_diff'], trial_info['var4_diff']]
        df['diff'] =[diff_list[int(r-1)][i] if math.isnan(r) == False else np.nan for i, r in enumerate(df['Response'])]
        
        # remove unnecessary columns to redce clutter 
        df = df.drop(columns=['correct_response', 'var1_diff', 'var2_diff', 'var3_diff', 'var4_diff'])
        frames.append(df)